In [1]:
import logging
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

In [2]:
class SoftmaxLayer(nn.Module):
    
    """Naive softmax-layer"""
    
    def __init__(self, output_dim, n_class):
        super(SoftmaxLayer, self).__init__()
        self.hidden2tag = nn.Linear(output_dim, n_class)
        self.criterion = nn.CrossEntropyLoss(size_average=False)
    
    def forward(self, x, y):
        tag_scores = self.hidden2tag(x)
        return self.criterion(tag_scores, y)

In [3]:
class SampledsoftmaxLayer(nn.Module):
    def __init__(self, output_dim, n_class, n_samples, use_cuda):
        super(SampledsoftmaxLayer, self).__init__()
        self.n_samples = n_samples
        self.n_class = n_class
        self.use_cuda = use_cuda
        self.criterion = nn.CrossEntropyLoss(size_average=False)
        self.negative_samples = []
        self.word_to_column = {0: 0}

        self.all_word = []
        self.all_word_to_column = {0: 0}

        self.column_emb = nn.Embedding(n_class, output_dim)
        self.column_emb.weight.data.uniform_(-0.25, 0.25)

        self.column_bias = nn.Embedding(n_class, 1)
        self.column_bias.weight.data.uniform_(-0.25, 0.25)

        self.oov_column = nn.Parameter(torch.Tensor(output_dim, 1))
        self.oov_column.data.uniform_(-0.25, 0.25)
        
    def forward(self, x, y):
        if self.training:
            for i in range(y.size(0)):
                y[i] = self.word_to_column.get(y[i].tolist())
            samples = torch.LongTensor(len(self.word_to_column)).fill_(0)
            for word in self.negative_samples:
                samples[self.word_to_column[word]] = word
        else:
            for i in range(y.size(0)):
                y[i] = self.all_word_to_column.get(y[i].tolist(), 0)
            samples = torch.LongTensor(len(self.all_word_to_column)).fill_(0)
            for word in self.all_word:
                samples[self.all_word_to_column[word]] = word
        
        if self.use_cuda:
            samples = samples.cuda()
            
        tag_scores = (x.matmul(self.embedding_matrix)).view(y.size(0), -1) + \
                     (self.column_bias.forward(samples)).view(1, -1)
        return self.criterion(tag_scores, y)
    

In [4]:
a = SampledsoftmaxLayer(1,1,1,1)

C:\Users\jinma\AppData\Local\Continuum\anaconda3\envs\basic\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
